#### CSCE 670 :: Information Storage and Retrieval :: Texas A&M University :: Spring 2017

#### PROJECT

# SCUPER: Social Circle and User PErsonality based Recommendation System

### Team : Aniket Bonde, Nagaraj Janakiraman, Sudheer Dantuluri

### [25% of final grade]

### Due: Tuesday, May 2 by 11:59pm

*Goal of this project:* Get hand-on experience with building a recommendation system.


# 1. Introduction and Problem Statement

 Recommender systems (RS) play a pivotal role in boosting the sales of major e-commerce systems. With rapid increase in the number of registered users and products on e-commerce systems, the problem of cold start for users (new users into the RS with little historical behavior) and the sparsity of datasets (the proportion of rated user-item pairs in all the user-item pairs of RS) has become increasingly intractable.

 Fortunately, the appearance of web2.0 greatly improved user’s initiative on the Internet, and then brought volume of social networks such as Facebook, Twitter, Yelp, Douban, Epinions, etc. This new factor of social network - the interpersonal interest based on circles of friends brings opportunities to the recommender system to solve the cold start & sparsity problem of datasets.

 In this Project, we propose to implement a personalized recommendation model that fuses in user’s personal interest and social factors like interpersonal similarity and interpersonal influence that is based on social network. The factor of user personal interest captures direct connection between user and item latent feature vectors, and social factors capture connections between user and his/here friend's latent feature vectors.

# 2. Related Work

 Recently, there have been some works that address to solve the cold start problem by factoring in social network features. Some recent significant contributions are CircleCon Model [1], ContextMF model [2] and [3]. [1] proposed a model that uses the concept of ‘inferred trust circle’ based on the domain-obvious circles of friends on social network. However, [2] demonstrated that individual preference (personal interest) is also a significant factor in social network. [3] builds on these ideas and proposes a personalized recommendation model based on probabilistic matrix factorization that considers three factors - personal interest, interpersonal interest similarity and interpersonal influence. Detailed explanation for each factor is presented in the high-level-approach section.

 In our project, we desire to understand the effects of factors proposed in [3], and evaluate it on a real dataset like ‘Yelp’ to recommend businesses for users with very less rating history (cold start scenario). We evaluate the model that considers user personal interest and social circle and compare it against a model like classic Matrix Factorization that neglects factors from social network.
        
### 2.1 References

[1] X.-W. Yang, H. Steck, and Y. Liu, “Circle-based recommendation in online social networks,” in Proc. 18th ACM SIGKDD Int. Conf. KDD, New York, NY, USA, Aug. 2012, pp. 1267–1275.

[2] M. Jiang et al., “Social contextual recommendation,” in Proc. 21st ACM Int. CIKM, New York, NY, USA, 2012, pp. 45–54.

[3] H. Feng and X. Qian, “Recommendation via user’s personality and social contextual,” in Proc. 22nd ACM CIKM, New York, NY, USA, 2013

[4] R. Sinha and K. Swearingen, “Comparing recommendations made by online systems and friends,” in Proc. DELOS Workshop Personalisation Recommender Systems Digital Libraries, Dublin, Ireland, 2001.

# 3. High-level Approach

   We take a Latent Factor Model based approach to build a personalized recommender system. The task of a recommender system is to decrease the error of predicted value and the actual rating.
   
   We first start with the basic matrix factorization model without any social factors. Thus, this basic model is trained on the observed rating data by minimizing the objective error function using gradient descent.
 

  We then add three factors proposed in [1], that factors the social interation informaation into the model for better personalization:

1. **Interpersonal influence: Whom you would trust?** 

  Interpersonal influence is the measure of *trust a particular user puts in another user*. The results in [4] show that the user’s friends consistently provided better recommendations. For example, 90% of users believe the book recommended is good from friends, 75% of users believe that the recommendation is useful from friends.
  
2. **Interpersonal interest similarity: Whose interest is similar to yours?**

 The basic idea is that *user latent feature vector should be similar to his/her friends’ latent feature vector* based on the similarity of their interest. Also, for cold start users, we infer interest circle to enhance the intrinsic link of user latent feature.

3. **User personal interest: Which items you would interest in?**
 
 Personal interest denotes *user’s individuality of rating items*, especially for the experienced users, and these factors fuse together to improve the accuracy and applicability of recommender system.
 
 ![title](PRM.jpg)
 
The above figure from [1] illustrates the three factors used in our model. 

   The model that includes the 3 factors is again trained on the training dataset to minimize an objective cost (see details in section 4.2.3) function using gradient descent to learn the best set of latent features.
 
   
   We then use the latent features to predict the ratings for the testing set and use RMSE and MAE as our evaluation metrics to comapare the performances.   
 
### Flow-chart


![title](flowchart.png)

The above flowchart depicts our high level overview. The flow is self-explanatory and hence we just discuss some important points.

* The training data is composed of both the user ratings and the social graph.
* We extract Personal Interests (PI) factor from the user ratings matrix, Interpersonal Influence (II) factor from the graph, and the Interpersonal interests Similarity (IS) from both the user ratings matrix and social interactions.
* The extracted features are inputed to a Personal Recommendation Model (PRM) algorithm (described in section 4.2) to learn the latent features that minimize the designed objective function.
* The latent features are used to predict the ratings on the test dataset and the metrics like RMSE/MAE are compared and analyzed.


# 4. Methodology

## 4.1 Dataset

Initially, we chose to use the [yelp](https://www.yelp.com/dataset_challenge) data-set  in our proposal. We started our project by trying to come up with a simplified dataset first. Couple of strategies that we tried include:

a. Finding user with maximum number of friends and then expanding the user-database by crawling breadth first to n(initialized to 1) levels from this user. We stop incrementing n if the size of the accumulated number of users crosses our target number of users. Then we consider only reviews and businesses that correspond to these users only.

b. Pick target/10 random users and expand them one level.Then we consider only reviews and businesses that correspond to these users only.

Code for the above experiments present in github @ dataset/baby/compressDataset.py and read_json.py.

However, while evaluating various strategies in trying to come up with a smaller, "proper" (one that preserves the social relations while trimming the dataset), we came across [this](http://smiles.xjtu.edu.cn/Download/Download_yelp.html) processed dataset. It is composed from the original yelp data-set by the very authors of the research paper that this project is based upon.

The authors have created these processed datasets for the following eight categories : restaurants, shopping, nightlife, pets, hotelstravel, homeservices, beautysvc and active. Among those, we chose the dataset "Night life" as we found the size of that dataset to be well suited for our task given the constraint on the resources (time and computational-power).



## 4.2 Algorithm

### 4.2.1 Notation

The following table provides the list of notations we use in this report and their significance.

| Symbol | Meaning |
|--|-------------------------------|
| $D$ | the topic distribution vector |
| $Q$ | the relevance matrix of users $u$'s interest to the topic of item $i$ |
| $W$ | the interest similarity matrix of users $u$ to user $v$|
| $S$ | the matrix of user $u$ trust on $v$|
| $H$ | the set of items rated by users|
| $U$ | user latent feature vector matrix|
| $P$ | item latent feature vector matrix|


### 4.2.2 Read dataset

In [5]:
import pandas as pd
import re
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
import warnings
warnings.filterwarnings("ignore")
import time
import scipy
from sklearn.preprocessing import normalize
from itertools import cycle
import sys
import pickle

items_path = r'one_category_dataset/items.txt'
users_path = r'one_category_dataset/users.txt'
ratings_train_path = r'one_category_dataset/nightlife_training.txt'
ratings_test_path = r'one_category_dataset/nightlife_test.txt'
subcategories_path = r'one_category_dataset/subcategories_list.txt'


eps = 10               # epsilon 

#mode = 0               # Simple MF model (No social graph)  
#mode = 1               #  UI (Q) + MF
#mode = 2               #  II (S) + MF
mode = 3               #  IS (W) + MF
# mode = 4               #  UI + II  (Q,S)
# mode = 5               #  UI + IS  (Q,W)
# mode = 6               #  II + IS  (S,W)
# mode = 7               #  ALL   (Q,S,W)

#print 'Mode:', mode

items_df = pd.read_table(items_path, delimiter = r'::', engine='python')
users_df = pd.read_table(users_path, delimiter = r':', engine='python')

#ratings_df = pd.read_table(ratings_path, delimiter = r'::', engine='python')

ratings_train_df = pd.read_table(ratings_train_path, delimiter = r'::', engine='python')
ratings_test_df = pd.read_table(ratings_test_path, delimiter = r'::', engine='python')

items = len(items_df)
users = len(users_df)

f = open(subcategories_path)
subcategories = [x.strip() for x in f.readlines()]
                 
def friends_str2list(str_friends):
    return map(int, re.sub(r'{', '', str_friends).split(',')[:-1])

def space2underscore(string):
    return re.sub(r' ', '_', string)
    
def topic_dist(sub_category_list, categories):
    tpc_dist = [0]*len(categories)
    for i in range(len(tpc_dist)):
        if (categories[i] in sub_category_list):
            tpc_dist[i] = 1
    return tpc_dist
       
    
users_df['friends'] = map(friends_str2list, users_df['friends'])
subcategories = map(space2underscore, subcategories)
items_df['sub_category'] = map(space2underscore, items_df['sub_category'])
items_df['topic_distribution'] = map(topic_dist, items_df['sub_category'], [subcategories]*(len(items_df['sub_category'])))
#users_df['topic_distribution'] = [[0]*18]*len(users_df['topic_distribution'])

nil_tpc_dist_items = []
for i in range(len(items_df['topic_distribution'])):
    if(items_df['topic_distribution'][i] == [0] * 18):
        nil_tpc_dist_items.append(i)



Mode: 3


### 4.2.3 Build Matrices

In [6]:
#Building Matrices
#m = number of users, n = number of items

temp, H = [], []
for user, group in ratings_train_df.groupby('user_id'):
    temp.append(np.mean(list(items_df.loc[list(group['item_id'])]['topic_distribution']), axis = 0))
    H.append(len(group))
    
users_df['topic_distribution'] = temp

#H = np.matrix(H[0])        
H = normalize(H[0], norm ='l1')


# 1) Q (m * n) = Relevance matrix of user 'u' to topic of item 'i'

Q = np.nan_to_num(1 - scipy.spatial.distance.cdist(list(users_df['topic_distribution']), list(items_df['topic_distribution']), 'cosine'))



# 2) S (m * m) = Trust of user 'u' on user 'v'
S = np.zeros((users, users))
S_sym = np.zeros((users, users))

for i in range(users):
    for user, friend in zip([i] * len(users_df['friends'][i]), users_df['friends'][i]):
        S[user, friend] = 1
        S_sym[user,friend] = 1
        S_sym[friend,user] = 1

# 3) W (m * m) = Similarity matrix of user 'u' to topic of user 'v'
W = np.nan_to_num(1 - scipy.spatial.distance.cdist(list(users_df['topic_distribution']), list(users_df['topic_distribution']), 'cosine'))
W = np.multiply(S_sym,W)  
W = normalize(W, norm='l2')  

R = np.empty([len(users_df), len(items_df)], dtype = np.float16)
I = np.empty([len(users_df), len(items_df)], dtype = np.float16)

R_test = np.empty([len(users_df), len(items_df)], dtype = np.float16)
I_test = np.empty([len(users_df), len(items_df)], dtype = np.float16)


num_ratings_train = 0;        
for user, item, rtng in zip(ratings_train_df['user_id'], ratings_train_df['item_id'], ratings_train_df['rating']):
    R[user, item] = rtng
    I[user, item] = 1
    num_ratings_train +=1;

    
num_ratings_test = 0;
for user, item, rtng in zip(ratings_test_df['user_id'], ratings_test_df['item_id'], ratings_test_df['rating']):
    R_test[user, item] = rtng
    I_test[user, item] = 1
    num_ratings_test +=1;
###############################################################################


### 4.2.3 Cost function & Optimization

$\psi (R, U, P, S, W, Q) = \psi_{\text{MF}} + \psi_{\text{II}} + \psi_{\text{IS}} + \psi_{\text{UI}} $

$\psi_{\text{MF}} = \frac{1}{2} \sum_{u,i}{(R_{u,i} - \hat{R}_{u,i})^2} + \frac{\lambda}{2} \left( ||U||_{F} + ||P||_{F}  \right)  $

$\psi_{\text{II}} = \frac{\beta}{2} \sum_{u}{\left( \left( U_u - \sum_v S_{u,v} U_v \right) \left( U_u - \sum_v S_{u,v} U_v \right)^T \right)}$

$\psi_{\text{IS}} = \frac{\gamma}{2} \sum_{u}\left( \left( U_u - \sum_v W_{u,v} U_v \right) \left( U_u - \sum_v W_{u,v} U_v \right)^T \right)$

$\psi_{\text{UI}} = \frac{\eta}{2} \sum_{u,i} |H_u|  \left( Q_{u,i} - U_u {P_i}^T \right)^2 $

### 4.2.8 Performing gradient descent (no vectorization)

$ \frac{\partial \psi}{\partial U_u} = \frac{\partial \psi_{\text{MF}}}{\partial U_u}   + \frac{\partial \psi_{\text{II}}}{\partial U_u} + \frac{\partial \psi_{\text{IS}}}{\partial U_u} + \frac{\partial \psi_{\text{UI}}}{\partial U_u} $


$ \frac{\partial \psi_{\text{MF}}}{\partial U_u} = \sum \limits_{i \in H_u}{ I_{u,i}\left( R_{u,i} - \hat{R}_{u,i} \right) P_i} + \lambda U_u $


$ \frac{\partial \psi_{\text{II}}}{\partial U_u} = \beta \left( U_u - \sum \limits_{v \in F_v} S_{u,v} U_v \right) - \beta \sum \limits_{v: u \in F_v} S_{v,u} \left( U_v - \sum \limits_{w \in F_v} S_{v,w} U_w \right)$


$ \frac{\partial \psi_{\text{IS}}}{\partial U_u} = \gamma \left( U_u - \sum \limits_{v \in F_u} W_{u,v} U_v \right) - \gamma \sum \limits_{v: u \in F_v} W_{v,u} \left( U_v - \sum \limits_{w \in F_v} W_{v,w} U_w \right)$


$\frac{\partial \psi_{\text{UI}}}{\partial U_u} = \eta \sum \limits_{i \in H_u} I_{u,i} |H_u|  \left(  U_u {P_i}^T - Q_{u,i}\right) P_i $


$\frac{\partial \psi}{\partial P_i} = \sum \limits_{u} I_{u,i} \left( \hat{R}_{u,i} - R_{u,i} \right) U_u + \lambda P_i + \eta \sum \limits_{u} I_{u,i} | H_u | \left( U_u {P_i}^T - Q_{u,i} \right) U_u $




### 4.2.9 Performing gradient descent (vectorization)

In [9]:
#Gradient Descent

#Parameters for Gradient Descent
global lamda, beta, gamma, eta, l
k = 10                                  #dimension of latent space
lamda = 0.1
beta = 30
gamma = 30
eta = 30
l = 0.000005

np.random.seed(0)
U = 0.1 * np.random.randn(users, k)
P = 0.1 * np.random.randn(items, k)

######################################################
#U = pickle.load( open( "U_0.pckl", "rb" ) )
#P = pickle.load( open( "P_0.pckl", "rb" ) )
########################################################

#r = np.mean(ratings_df['rating'])
r = np.empty([len(users_df), len(items_df)], dtype = np.float16)
for user, rating_group in ratings_train_df.groupby('user_id'):
    r[user][:] = np.mean(rating_group['rating'])
###############################################################################

#Gradient Descent

def cal_pred_rating(r, U, P):
    return (r + np.matmul(U, P.transpose()))
    
def cal_error_der_P(I_, R_, U_, H_, Q_, P_):
    first_fac = 0
    second_fac = 0
    third_fac = 0
    
    first_fac = np.matmul(np.multiply(I_.transpose(), R_.transpose()), U_)
    
    second_fac = lamda * P_
    
    if(mode in [1,4,5,7]):
        third_fac = eta * np.matmul(np.multiply(np.multiply(I_.transpose(), np.matlib.repmat(H_, items, 1)), 
                             (np.subtract((np.matmul(U_, P_.transpose())), Q_)).transpose()), U_)
    
    return first_fac + second_fac + third_fac


R_cap = cal_pred_rating(r, U, P)

#print 'Rcap:', R_cap

def cal_error_der_U(I_, R_, P_, H_, Q_, U_, W_, S_):
    
    first_fac = 0
    second_fac = 0
    third_fac = 0
    fourth_fac = 0
    fifth_fac = 0
    sixth_fac = 0
    seventh_fac = 0
    
 
    first_fac = np.matmul(np.multiply(I_, R_), P_)
    
    second_fac = lamda * U_
    
    if(mode in [2,4,6,7]):
        third_fac = beta * np.subtract(U_, np.matmul(S_,U_))
 
    if(mode in [2,4,6,7]):
        fourth_fac = -beta *  np.matmul(S_.transpose(),np.subtract(U, np.matmul(S,U))) 
    
    if(mode in [3,5,6,7]):
        fifth_fac = gamma * np.subtract(U_, np.matmul(W_,U_))
    
    if(mode in [3,5,6,7]):
        sixth_fac = -gamma * np.matmul(W_.transpose(),np.subtract(U, np.matmul(W,U)))
    
    if(mode in [1,4,5,7]):
        seventh_fac = eta * np.matmul(np.multiply(np.multiply(I_, (np.matlib.repmat(H_, items, 1)).transpose()), 
                             np.subtract((np.matmul(U_, P_.transpose())), Q_)), P_)
        
    return first_fac + second_fac + third_fac + fourth_fac + fifth_fac + sixth_fac +seventh_fac
    
#error_der_U = map(cal_error_der_U, I, (R_cap - R), [P] * users, H.transpose(), Q, U)

def cal_error_fn(R, R_cap, H, Q, U, P, S, W, I):
    
    first_fac = 0
    second_fac = 0
    third_fac = 0
    fourth_fac = 0
    fifth_fac = 0
    
    
    first_fac = np.sum(np.sum(np.multiply(R - np.multiply(R_cap,I), R - np.multiply(R_cap,I)), axis = 1), axis = 0) / float(2)
                      
    second_fac = (lamda/float(2)) *(np.linalg.norm(U,ord='fro') + np.linalg.norm(P,ord='fro'))
    
    if(mode in [2,4,6,7]):
        third_fac_temp = np.subtract(U, np.matmul(S,U))
        third_fac = (beta/float(2)) * ((np.linalg.norm(third_fac_temp,ord='fro'))**2)
    
    if(mode in [3,5,6,7]):
        fourth_fac_temp = np.subtract(U, np.matmul(W,U))
        fourth_fac = (gamma/float(2)) * ((np.linalg.norm(fourth_fac_temp,ord='fro'))**2)
    
    if(mode in [1,4,5,7]):
        fifth_fac_temp = np.subtract(Q, np.matmul(U, P.transpose()))
        fifth_fac = (eta/float(2)) * np.sum(np.sum(np.multiply((np.matlib.repmat(H, items, 1)).transpose(), np.multiply(fifth_fac_temp, fifth_fac_temp)), axis = 1), axis = 0)
    
        
    #print 'first:',first_fac , 'second:', second_fac, 'fifth:', fifth_fac
    
    return first_fac + second_fac + third_fac + fourth_fac + fifth_fac

#print(cal_error_fn(R, R_cap, H, Q, U, P, S,W,I))

U = U.astype(dtype = np.float16)
P = P.astype(dtype = np.float16)

t=0

identifier = (np.arange(users)).transpose() 

Error_train_old = 100000
Error_test_old = 100000

while(t<10000):
        print 'iter:', str(t+1)
        error_der_U = cal_error_der_U(I, (R_cap - R), P, H, Q, U, W, S)        
        error_der_P = cal_error_der_P(I, (R_cap - R), U, H, Q, P)
        
        U = np.subtract(U, np.multiply(l, error_der_U))
        
        for i in range(users):
            for j in range(k):
                P[i][j] -= (l * error_der_P[i][j])
                
        R_cap = cal_pred_rating(r, U, P)
        
         
        RMSE_test = np.sqrt(np.sum(np.square(np.subtract(np.multiply(I_test,R_cap),R_test)))/float(num_ratings_test))
        RMSE_train = np.sqrt(np.sum(np.square(np.subtract(np.multiply(I,R_cap),R)))/float(num_ratings_train))
        
        
        print 'RMSE_train:', RMSE_train, 'RMSE_test', RMSE_test
        #print R_cap
        
        Error_train = cal_error_fn(R, R_cap, H, Q, U, P,S,W,I)
        Error_test = cal_error_fn(R_test, R_cap, H, Q, U, P,S,W,I_test)
        
        print 'Error_train:', Error_train ,'Error_test:', Error_test
        
        if t>0:
            
            if (Error_test-Error_test_old > 0):
                if (Error_train-Error_train_old > 0):
                    print 'Decrease learning rate!'
                    sys.exit(0)
                else:
                    print 'Exiting because of overfitting'
                #sys.exit(0)
                
            if (Error_train-Error_train_old > 0):
                print 'Decrease learning rate!'
                sys.exit(0)
                
            if ((Error_train_old-Error_train)<eps):
                print 'Converged!\n '
            #    sys.exit(0)    
                
        Error_train_old = Error_train
        Error_test_old = Error_test
        
        t +=1
    

iter: 1
RMSE_train: 0.968794374411 RMSE_test 1.02551161748
Error_train: 48239.9904255 Error_test: 22628.8232938
iter: 2
RMSE_train: 0.968793342935 RMSE_test 1.02551105057
Error_train: 48195.3761304 Error_test: 22584.2750363
iter: 3
RMSE_train: 0.968792314891 RMSE_test 1.02551049057
Error_train: 48151.8873687 Error_test: 22540.8522026
iter: 4
RMSE_train: 0.968791290125 RMSE_test 1.0255099373
Error_train: 48109.4789913 Error_test: 22498.509651
iter: 5
RMSE_train: 0.968790268548 RMSE_test 1.02550939056
Error_train: 48068.1078515 Error_test: 22457.2042376
iter: 6
RMSE_train: 0.968789250069 RMSE_test 1.02550885015
Error_train: 48027.7327085 Error_test: 22416.8947239
iter: 7
RMSE_train: 0.96878823461 RMSE_test 1.0255083159
Error_train: 47988.3141397 Error_test: 22377.5416895
iter: 8
RMSE_train: 0.968787222086 RMSE_test 1.02550778763
Error_train: 47949.8144561 Error_test: 22339.1074481
iter: 9
RMSE_train: 0.968786212501 RMSE_test 1.02550726517
Error_train: 47912.19763 Error_test: 22301.555967

# 5. Evaluation
 
## 5.1 Metric

 We evaluate our model by comparing the two models (with and without social interaction information) based on Root Mean Squared Error (RMSE) and Mean Absolute Error (MAE). We also evaluate our models for users that have very less rating history (cold start scenario) and draw conclusions on the effect of using social features.
 
## 5.2 Results

### 5.2.1 PRM Model - Iteration vs training-error & test-error

Graph-plot Curve

Write Key-takeaways

### 5.PRM Model - Bar graph comparing RMSE of MF, UI, IS, II, PRM

bar-graph

Write Key-takeaways

### PRM Model - Bar graph comparing MAE of MF, UI, IS, II, PRM

bar-graph

Write Key-takeaways

      

# 5. Conclusion and Next Steps

        

In this poject, a personalized recommendation approach was implemented by combining social network factors: personal interest, interpersonal interest similarity, and interpersonal influence. In particular, the personal interest denotes user’s individuality of rating items, especially for the experienced users, and these factors were fused together to improve the accuracy and applicability of recommender system. 


Experiments were performed on yelp "Night-life" processed dataset and it was observed that the model that fuses in social network information along with users personal interest shows significant improvements over approaches that
do not factor in social network information. 

## Problems Faced/ Lessons Learnt